In [38]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import matplotlib.pylab as plt

In [39]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [40]:
# Import des questions

In [168]:
answers_grand_debat_DEC_raw = pd.read_csv('raw/large_datasets_ELF/DEMOCRATIE_ET_CITOYENNETE.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_FDP_raw = pd.read_csv('raw/large_datasets_ELF/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv', encoding="utf-8").drop_duplicates(keep=False)

In [191]:
def keep_popular_hashtags_by_question(dataframe):
    for question in dataframe['id_elf'].unique():    
        allowed_categories_tuples = Counter(dataframe[dataframe['id_elf'] == question]['Categorie']).most_common(20)
        allowed_categories = [element[0] for element in allowed_categories_tuples]
        dataframe = dataframe[(dataframe['id_elf'] != 'question') | (dataframe['Categorie'].isin(allowed_categories))]
    return dataframe

def process_LGD_dataset(raw_dataframe):
    """
    """
    answers_annotees = pd.read_csv('raw/actions.csv', encoding="utf-8").rename(columns={'Contribution': 'reference'})
    raw_dataframe = pd.merge(answers_annotees, raw_dataframe, on='reference')
    print(raw_dataframe.shape)
    
    questions_dictionary = pd.read_csv('raw/match_questions_ELF_LGA.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, questions_dictionary, left_on='Question', right_on='id_gda')
    print(raw_dataframe.shape)

    gda_dictionary = pd.read_csv('raw/matching_gda_id1_id2.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, gda_dictionary, left_on='id_gda', right_on='id1_gda')
    print(raw_dataframe.shape)
    
    raw_dataframe["corpus"] = None
    
    for index, row in raw_dataframe.iterrows():
        raw_dataframe.at[index,'corpus'] = raw_dataframe.loc[index][raw_dataframe.filter(like=row['id2_gda']).columns[0]]

    raw_dataframe = raw_dataframe[['reference', 'id_elf', 'corpus', 'Categorie', 'Annotateur']]
    
    removed_categories = ['Z Autres', 'Sans réponse / Hors sujet / Inclassable']
    raw_dataframe = raw_dataframe[~raw_dataframe['Categorie'].isin(removed_categories)]
    print(raw_dataframe.shape)
    
    raw_dataframe = keep_popular_hashtags_by_question(raw_dataframe)
    print(raw_dataframe.shape)
    
    raw_dataframe = pd.DataFrame({'count' : raw_dataframe.groupby( ['reference', 'Categorie', 'corpus', 'id_elf'] ).apply(lambda x: x.Annotateur.nunique())}).reset_index()
    print(raw_dataframe.shape)
    
    return raw_dataframe
    

In [196]:
dataframe = process_LGD_dataset(answers_grand_debat_DEC_raw)

(56677, 53)
(5918, 55)
(5918, 57)
(4679, 5)
(4679, 5)
(1883, 5)


In [197]:
dataframe.head()

,reference,Categorie,corpus,id_elf,count
0,1-10005,Y Rien,nsp,DC13,1
1,1-10092,Aucun financement par l'Etat,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,1
2,1-10092,Garantir la liberté de culte,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,1
3,1-10092,Loi de la République avant celles de la religion,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,5
4,1-10092,Lutte contre extrémismes,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,2


In [198]:
answers_grand_debat_input = dataframe.rename(columns={'reference': 'contribution', 'Categorie': 'tag'})
answers_grand_debat_input.to_csv('preprocessed/reponses_annotees_multilabel_FDP.csv', sep='\t', encoding='utf-8')

In [199]:
answers_grand_debat_input.head()

,contribution,tag,corpus,id_elf,count
0,1-10005,Y Rien,nsp,DC13,1
1,1-10092,Aucun financement par l'Etat,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,1
2,1-10092,Garantir la liberté de culte,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,1
3,1-10092,Loi de la République avant celles de la religion,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,5
4,1-10092,Lutte contre extrémismes,1/. LA FERMETÉ; la voix du peuple est supérieu...,DC13,2
